<a href="https://colab.research.google.com/github/AravaChoudhary/ML-Pipeline-/blob/main/pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [24]:
df = sns.load_dataset('tips')
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [25]:
# predict what's the time ? lunch or dinner --> time is the target variable
df.time.unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [26]:
# EDA --> SUBJECTIVE
# encoding, missing value treatment , scaling --> automate
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [27]:
# since time is a nominal variable we will use label encoder
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['time'] = encoder.fit_transform(df['time'])
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,0,3
240,27.18,2.00,Female,Yes,Sat,0,2
241,22.67,2.00,Male,Yes,Sat,0,2
242,17.82,1.75,Male,No,Sat,0,2


In [28]:
df.time.unique() # dinner --> 1, lunch --> 0

array([0, 1])

In [29]:
X = df.drop('time' , axis = 1)
y = df['time']

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 1)

In [31]:
# Handling the missing values
# data encoding
# feature scaling

from sklearn.impute import SimpleImputer # for missing values
from sklearn.preprocessing import OneHotEncoder # for Data Encoding
from sklearn.preprocessing import StandardScaler # for Scaling

In [32]:
from sklearn.pipeline import Pipeline # A sequence of Data Transformer
from sklearn.compose import ColumnTransformer

In [33]:
num_cols = ['total_bill','tip','size']
cat_cols = ['sex','smoker','day']


#feature engineering automation using pipeline and column transformer
num_pipeline = Pipeline(
    steps = [
              ('imputation',SimpleImputer(strategy = 'median')),
              ('scaling',StandardScaler())
            ]
)

cat_pipeline = Pipeline(
    steps = [
              ('imputation',SimpleImputer(strategy = 'most_frequent')),
              ('encoding',OneHotEncoder())
            ]
)

In [34]:
col_trans = ColumnTransformer([('num_pipeline', num_pipeline, num_cols),
                               ('cat_pipeline', cat_pipeline, cat_cols)])

In [35]:
X_train = col_trans.fit_transform(X_train)
X_test = col_trans.transform(X_test)

In [36]:
X_test

array([[-1.85376383, -1.48209775, -1.60171757,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         0.        ],
       [-0.08453291,  0.04984713, -0.57766863,  1.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.79501474,  0.36389583,  0.44638031,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         0.        ],
       [-0.59356688, -0.33313909, -0.57766863,  0.        ,  1.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.18349826,  0.04984713, -0.57766863,  0.        ,  1.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [-1.32783714, -1.14506988, -0.57766863,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         0.   

In [39]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

models = {'Support Vector Classifier' : SVC(),
          'DT Classifier' : DecisionTreeClassifier()}

In [46]:
def model_train_eval(X_train, y_train, X_test, y_test, models):
  evaluation = {}

  for i in range(len(models)):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    model_score = accuracy_score(y_test , y_pred)
    evaluation[list(models.keys())[i]] = model_score

  return evaluation

In [47]:
model_train_eval(X_train, y_train, X_test, y_test, models)

{'Support Vector Classifier': 0.9183673469387755,
 'DT Classifier': 0.9387755102040817}